This notebook creates the tables: 
* station_id_lat_lon
* station_distance


station_id_lat_lon columns:
* station	
* station_id	
* latitude	
* longitude

station_distance:
* start station id	
* end station id	
* miles	

Note:
* station_id_lat_lon: Coordinates attained from the latest trip information from citibike site

* data source: https://s3.amazonaws.com/tripdata/index.html
* Most recent file: 202011-citibike-tripdata.csv
 
* station_distance: geodestic distance between the two coordinates
* reference on geodestic distnace: https://en.wikipedia.org/wiki/Geodesics_on_an_ellipsoid


In [1]:
import numpy as np
import pandas as pd

In [2]:
tripdata_raw = pd.read_csv('/Users/bkuo/Local Documents/Bootcamp 2020/Project/Capstone/CitiBikeCapStone/Data/Trip data/202011-citibike-tripdata.csv')

In [3]:
tripdata_raw.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
count,1.736704e+06,1.736704e+06,1.736704e+06,1.736704e+06,1.736704e+06,1.736704e+06,1.736704e+06,1.736704e+06,1.736704e+06,1.736704e+06
mean,1.150961e+03,2.134801e+03,4.074266e+01,-7.397583e+01,2.130156e+03,4.074232e+01,-7.397599e+01,3.918440e+04,1.981293e+03,1.181464e+00
std,1.155238e+04,1.555192e+03,3.627091e-02,2.348935e-02,1.556285e+03,3.617605e-02,2.356461e-02,9.057289e+03,1.259026e+01,5.982931e-01
min,6.100000e+01,7.200000e+01,4.065540e+01,-7.402535e+01,7.200000e+01,4.065540e+01,-7.407106e+01,1.452900e+04,1.884000e+03,0.000000e+00
25%,3.950000e+02,4.380000e+02,4.071817e+01,-7.399266e+01,4.370000e+02,4.071773e+01,-7.399301e+01,3.482200e+04,1.969000e+03,1.000000e+00
50%,7.060000e+02,3.126000e+03,4.074174e+01,-7.398068e+01,3.124000e+03,4.074146e+01,-7.398093e+01,4.132000e+04,1.984000e+03,1.000000e+00
75%,1.256000e+03,3.532000e+03,4.076591e+01,-7.395961e+01,3.530000e+03,4.076527e+01,-7.395995e+01,4.691100e+04,1.991000e+03,2.000000e+00
max,2.422830e+06,4.249000e+03,4.085225e+01,-7.388431e+01,4.249000e+03,4.085225e+01,-7.388431e+01,4.993200e+04,2.004000e+03,2.000000e+00


In [4]:
tripdata_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1736704 entries, 0 to 1736703
Data columns (total 15 columns):
tripduration               int64
starttime                  object
stoptime                   object
start station id           int64
start station name         object
start station latitude     float64
start station longitude    float64
end station id             int64
end station name           object
end station latitude       float64
end station longitude      float64
bikeid                     int64
usertype                   object
birth year                 int64
gender                     int64
dtypes: float64(4), int64(6), object(5)
memory usage: 198.8+ MB


In [5]:
tripdata_raw.isna().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

Following cells makes the stations lat lon table:

In [19]:
# finding distinct station lat lons
s_lat_lon = tripdata_raw[['start station name', 'start station id','start station latitude', 'start station longitude']]
s_lat_lon = s_lat_lon.drop_duplicates().rename(columns = {'start station name':'station', 'start station id':'station_id','start station latitude':'latitude', 
                            'start station longitude':'longitude'})
e_lat_lon = tripdata_raw[['end station name', 'end station id', 'end station latitude', 'end station longitude']]
e_lat_lon = e_lat_lon.drop_duplicates().rename(columns = {'end station name':'station', 'end station id':'station_id','end station latitude':'latitude', 
                            'end station longitude':'longitude'})
stn_lat_lon = pd.concat([s_lat_lon, e_lat_lon]).drop_duplicates()


In [20]:
# total stations: 1161 stations (not considering active or not)
stn_lat_lon.shape

(1161, 4)

In [21]:
# final table
stn_lat_lon.head()

,station,station_id,latitude,longitude
0,W Broadway & Spring St,3467,40.724947,-74.001659
1,40 Ave & 9 St,3557,40.757420,-73.945133
2,Roebling St & N 4 St,3085,40.714690,-73.957390
3,Cliff St & Fulton St,3783,40.708380,-74.004950
4,W 59 St & 10 Ave,422,40.770513,-73.988038


In [11]:
stn_lat_lon.to_csv('station_id_lat_lon')

Following cells calculates distances between all stations:

In [22]:
# creating OD pair distance table
s_lat_lon['key'] = 1
e_lat_lon['key'] = 1

In [23]:
stn_dist = pd.merge(s_lat_lon, e_lat_lon, on = 'key').drop("key", 1).drop_duplicates()  #1161
stn_dist.shape

(1337472, 8)

In [ ]:
# running this part of the query will take a very very long while
from geopy import distance

stn_dist['start_coor'] = list(zip(stn_dist.latitude_x, stn_dist.longitude_x)) 
stn_dist['end_coor'] = list(zip(stn_dist.latitude_y, stn_dist.longitude_y))

dist = []
for start in stn_dist['start_coor']:
    for end in stn_dist['end_coor']:
        dist.append([start, end, distance.distance(start, end).miles])    
dist = pd.DataFrame(dist, columns = ['start_coor', 'end_coor', 'miles'])   
stn_dist = stn_dist.merge(dist, left_on = ['start_coor', 'end_coor'], \
                          right_on= ['start_coor', 'end_coor'], how='left')
stn_dist = stn_dist.drop_duplicates()
stn_dist = stn_dist[['station_id_x', 'station_id_y', 'miles']]
stn_dist = stn_dist.rename(columns = {'station_id_x':'s_station_id',
                            'station_id_y':'e_station_id'})


In [ ]:
stn_dist.head()

In [ ]:
stn_dist.to_csv('station_distance')